In [1]:
from dotenv import load_dotenv
load_dotenv()

import os
import json
import re
from typing import TypedDict, List, Dict, Optional
from langgraph.graph import StateGraph, END
import google.generativeai as genai
from tavily import TavilyClient


In [2]:
gemini_api_key = os.getenv("GEMINI_API_KEY")
tavily_api_key = os.getenv("TAVILY_API_KEY")


In [3]:

if not gemini_api_key:
    raise ValueError("GEMINI_API_KEY not found in environment variables!")
genai.configure(api_key=gemini_api_key)

if not tavily_api_key:
    raise ValueError("TAVILY_API_KEY not found in environment variables!")
tavily = TavilyClient(api_key=tavily_api_key)


In [4]:

# ---- Helper Functions ----
def call_gemini(prompt: str, retries=3, delay=5) -> str:
    """Call Gemini API with retry logic."""
    for attempt in range(retries):
        try:
            model = genai.GenerativeModel("gemini-2.5-flash")  # Stable model
            response = model.generate_content(prompt)
            return response.text.strip() if response.text else ""
        except Exception as e:
            print(f"Gemini API error (attempt {attempt+1}/{retries}): {e}")
            if attempt < retries - 1:
                import time
                time.sleep(delay)
            else:
                return ""


In [5]:

def search_tavily(query: str) -> list[str]:
    """Return up to 5 Tavily search result snippets as a list of strings."""
    try:
        response = tavily.search(query=query, top_k=5)
        results = response.get("results", [])
        return [r.get("snippet") or r.get("content") or r.get("title", "") for r in results][:5]
    except Exception as e:
        print(f"Tavily search failed: {e}")
        return []

In [6]:

def safe_parse_json(response_text: str, fallback: Dict = None) -> Dict:
    """Safely parse JSON with fallbacks for empty/invalid responses."""
    if not response_text or not response_text.strip():
        print("Warning: Empty API response - using fallback.")
        return fallback or {"error": "Empty response"}
    try:
        return json.loads(response_text)
    except json.JSONDecodeError as e:
        print(f"JSON Parse Error: {e}")
        match = re.search(r"\{.*\}", response_text, re.DOTALL)
        if match:
            try:
                return json.loads(match.group(0))
            except json.JSONDecodeError:
                pass
        return fallback or {"error": f"Invalid response: {response_text[:100]}..."}


In [7]:

# ---- State Definition ----
class InterviewState(TypedDict):
    topic: str
    content: List[str]
    questions: List[str]
    answers: List[str]
    feedback: List[Dict]
    current_question: Optional[str]
    current_answer: Optional[str]
    step: int
    max_questions: int
    final_evaluation: Optional[Dict]
    messages: List[Dict]
    question_type: str  # "broad_followup", "narrow_followup", "broad_nonfollowup", "narrow_nonfollowup"


In [8]:

# ---- Nodes ----
def setup_node(state: InterviewState) -> InterviewState:
    print(" Welcome to the AI Interviewer (Question Evaluation Mode)!")
    topic = input("Enter the interview topic: ").strip()
    
    # Prompt for question type preference
    print("\nChoose question style:")
    print("1. Broad, follow-up questions (general, builds on previous answers)")
    print("2. Narrow, follow-up questions (specific, probes details from previous answers)")
    print("3. Broad, non-follow-up questions (general, new topic aspects)")
    print("4. Narrow, non-follow-up questions (specific, new topic aspects)")
    choice = input("Enter choice (1-4): ").strip()
    question_type_map = {
        "1": "broad_followup",
        "2": "narrow_followup",
        "3": "broad_nonfollowup",
        "4": "narrow_nonfollowup"
    }
    question_type = question_type_map.get(choice, "broad_followup")  # Default to broad_followup

    content_list = search_tavily(f"key areas for interview on: {topic}")
    initial_messages = [{"role": "user", "content": f"Interview topic: {topic}"}]

    # Generate the first question
    prompt_question = f"""
You are an expert interviewer. Using the following reference content:
{content_list}

Generate question #1 for the topic: {topic}.
{'Ask a broad, general question.' if question_type.startswith('broad') else 'Ask a specific, detailed question.'}
Return ONLY the question text.
"""
    first_question = call_gemini(prompt_question) or "Tell me about your interest in this topic."

    return {
        **state,
        "topic": topic,
        "content": content_list,
        "messages": initial_messages,
        "step": 0,
        "questions": [],
        "answers": [],
        "feedback": [],
        "current_question": first_question,
        "question_type": question_type
    }


In [9]:

def get_answer_node(state: InterviewState) -> InterviewState:
    """Get candidate answer"""
    current_q = state.get("current_question")
    if not current_q:
        raise ValueError("No current_question found in state.")

    print(f"\n❓ Generated Question {state['step'] + 1}: {current_q}\n")
    answer = input("💭 Your answer: ").strip()

    new_messages = state['messages'] + [
        {"role": "interviewer", "content": current_q},
        {"role": "candidate", "content": answer}
    ]

    return {
        **state,
        "current_answer": answer,
        "messages": new_messages,
        "questions": state['questions'] + [current_q],
        "answers": state['answers'] + [answer]
    }

def evaluate_question_node(state: InterviewState) -> InterviewState:
    """Rate the quality of the last question and the candidate's answer."""
    # Compile transcript of previous Q&A and feedback
    transcript = ""
    for i in range(len(state['questions']) - 1):
        q = state['questions'][i]
        a = state['answers'][i]
        f = state['feedback'][i] if i < len(state['feedback']) else {}
        transcript += f"Previous Q{i+1}: {q}\nPrevious A{i+1}: {a}\nPrevious Feedback: {f.get('question_feedback', {}).get('feedback', '')}\n\n"

    last_q = state['questions'][-1]
    last_a = state['answers'][-1]
    full_messages = json.dumps(state['messages'])
    full_content = "\n".join(state['content'])

    # Question feedback prompt
    question_prompt = f"""
You are an expert interviewer. Evaluate the following question for its clarity, relevance, and ability to probe understanding, considering the ENTIRE interview history, accumulated context, all previous messages, questions, answers, and feedback.

Full Interview History (Messages): {full_messages}
Accumulated Context (Search Snippets): {full_content}
Previous Q&A Transcript: {transcript}
Current Question: {last_q}
Current Candidate Answer: {last_a}

Provide a rating (1-10) for question quality and 2-3 sentence feedback. Consider how well this question builds on prior answers, avoids repetition, incorporates context, and advances the topic.
Return in JSON format:
{{
    "rating": 0,
    "feedback": "..."
}}
"""
    question_feedback_text = call_gemini(question_prompt)
    question_feedback = safe_parse_json(question_feedback_text, {"rating": 0, "feedback": "Failed to generate question feedback."})

    # Answer feedback prompt
    answer_prompt = f"""
You are an expert interviewer. Evaluate the following candidate answer for its clarity, relevance, depth, and alignment with the question, considering the ENTIRE interview history and context.

Full Interview History (Messages): {full_messages}
Accumulated Context (Search Snippets): {full_content}
Previous Q&A Transcript: {transcript}
Current Question: {last_q}
Current Candidate Answer: {last_a}

Provide a rating (1-10) for answer quality and 2-3 sentence feedback. Highlight strengths and areas for improvement.
Return in JSON format:
{{
    "rating": 0,
    "feedback": "..."
}}
"""
    answer_feedback_text = call_gemini(answer_prompt)
    answer_feedback = safe_parse_json(answer_feedback_text, {"rating": 0, "feedback": "Failed to generate answer feedback."})

    feedback = {
        "question_feedback": question_feedback,
        "answer_feedback": answer_feedback
    }
    print(f"💡 Question Feedback: {question_feedback['feedback']} (Rating: {question_feedback['rating']})")
    print(f"💡 Answer Feedback: {answer_feedback['feedback']} (Rating: {answer_feedback['rating']})")

    return {
        **state,
        "feedback": state['feedback'] + [feedback],
        "step": state['step'] + 1
    }

In [10]:

def generate_question_node(state: InterviewState) -> InterviewState:
    """Generate next question based on type (broad/narrow, follow-up/non-follow-up)."""
    updated_content = state['content']
    question_type = state['question_type']
    is_followup = "followup" in question_type
    is_broad = question_type.startswith("broad")

    if state['step'] > 0:
        last_q = state['questions'][-1]
        last_a = state['answers'][-1]
        tavily_results = search_tavily(f"{state['topic']} interview context: Q: {last_q} A: {last_a}")
        updated_content += tavily_results

    prompt_instruction = ""
    if is_followup:
        prompt_instruction = f"Generate a {'broad, general' if is_broad else 'specific, detailed'} follow-up question that directly probes details from the previous answer: {state['answers'][-1] if state['answers'] else ''}."
    else:
        prompt_instruction = f"Generate a {'broad, general' if is_broad else 'specific, detailed'} question that explores a new aspect of the topic, independent of the previous answer."

    prompt_question = f"""
You are an expert interviewer. Using the following reference content:
{updated_content}

{prompt_instruction}
Topic: {state['topic']}
Question number: {state['step'] + 1}
Return ONLY the question text.
"""
    question = call_gemini(prompt_question) or f"Tell me more about {state['topic']}."

    return {
        **state,
        "current_question": question,
        "content": updated_content
    }


In [11]:

def final_evaluation_node(state: InterviewState) -> InterviewState:
    """Generate final evaluation of all questions."""
    print("\n📊 Generating final evaluation of all questions...")
    transcript = ""
    for i, (q, a, f) in enumerate(zip(state['questions'], state['answers'], state['feedback']), 1):
        transcript += f"Q{i}: {q}\nA{i}: {a}\nQuestion Feedback: {f['question_feedback']['feedback']} (Rating: {f['question_feedback']['rating']})\nAnswer Feedback: {f['answer_feedback']['feedback']} (Rating: {f['answer_feedback']['rating']})\n\n"

    prompt = f"""
Based on this transcript, produce a JSON summary evaluation of the questions:
{transcript}

JSON format ONLY:
{{
    "overall_quality": 0-10,
    "strengths": ["..."],
    "areas_for_improvement": ["..."],
    "recommendation": "keep/revise/remove",
    "final_feedback": "..."
}}
"""
    response_text = call_gemini(prompt)
    evaluation = safe_parse_json(response_text, {"overall_quality": 0, "recommendation": "revise", "final_feedback": "Failed to generate evaluation."})

    return {**state, "final_evaluation": evaluation}


In [12]:

def display_results_node(state: InterviewState) -> InterviewState:
    """Display final report"""
    print("\n" + "="*60)
    print(" INTERVIEW COMPLETE - FINAL REPORT")
    print("="*60)
    print(f"\n Topic: {state['topic']}")
    print("\n Questions & Feedback:")
    for i, (q, a, f) in enumerate(zip(state['questions'], state['answers'], state['feedback']), 1):
        print(f"\n{i}. {q}")
        print(f"    Answer: {a}")
        print(f"    Feedback: {f}")

    print("\n📊 Final Evaluation:")
    eval_data = state['final_evaluation']
    if "error" in eval_data:
        print(" Could not parse evaluation:", eval_data["error"])
    else:
        for k, v in eval_data.items():
            print(f"   {k}: {v}")

    with open("interview_results.json", "w") as f:
        json.dump(state, f, indent=2)
    print("\n Results saved to 'interview_results.json'")
    return state


In [13]:

# ---- Graph ----
def should_continue(state: InterviewState) -> str:
    return "generate_question" if state['step'] < state['max_questions'] else "final_evaluation"


In [14]:

builder = StateGraph(InterviewState)
builder.add_node("setup", setup_node)
builder.add_node("get_answer", get_answer_node)
builder.add_node("evaluate_question", evaluate_question_node)
builder.add_node("generate_question", generate_question_node)
builder.add_node("final_evaluation", final_evaluation_node)
builder.add_node("display_results", display_results_node)

builder.set_entry_point("setup")
builder.add_edge("setup", "get_answer")
builder.add_edge("get_answer", "evaluate_question")
builder.add_conditional_edges(
    "evaluate_question",
    should_continue,
    {"generate_question": "generate_question", "final_evaluation": "final_evaluation"}
)
builder.add_edge("generate_question", "get_answer")
builder.add_edge("final_evaluation", "display_results")
builder.add_edge("display_results", END)

interview_graph = builder.compile()


In [ ]:

# ---- Run ----
if __name__ == "__main__":
    initial_state = {
        "topic": "",
        "content": [],
        "questions": [],
        "answers": [],
        "feedback": [],
        "current_question": None,
        "current_answer": None,
        "step": 0,
        "max_questions": 3,
        "final_evaluation": None,
        "messages": [],
        "question_type": "broad_followup"
    }
    final_state = interview_graph.invoke(initial_state)

 Welcome to the AI Interviewer (Question Evaluation Mode)!
